In [1]:
from tensorflow.python.keras.layers import Input,Dense,Reshape,Lambda,RNN,Concatenate,StackedRNNCells
from tensorflow.python.keras.models import Model,load_model
from tensorflow.python.keras.utils import multi_gpu_model
from tensorflow.python.keras.optimizers import Adam,SGD
from tensorflow.python.keras.callbacks import LearningRateScheduler,Callback,ModelCheckpoint,TensorBoard

import tensorflow as tf
import datetime
import os
import numpy as np
import pickle

%env PYTHONHASHSEED=0
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
os.environ["CUDA_VISIBLE_DEVICES"]="1"

/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/

env: PYTHONHASHSEED=0


/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site

In [2]:
from sklearn.model_selection import train_test_split
#features
n2v_start = np.load("start_emb_128.npy")
n2v_end = np.load("end_emb_128.npy")
lanes = np.load("lanes.npy")
oneway = np.load("oneway.npy")
roadtype = np.load("highway.npy")
maxspeed = np.load("maxspeed.npy")
nodetype = np.load("nodetype.npy")
lanes_df = np.load("lanes_df.npy")
#target
meanspeed = np.load("meanspeed_only.npy")
z_score = np.load('z-score.npy')
congestion = np.load('congestion.npy')

#split
n2v_start_train, n2v_start_test, n2v_end_train, n2v_end_test, lanes_train, lanes_test, oneway_train, oneway_test, roadtype_train, roadtype_test, maxspeed_train, maxspeed_test, nodetype_train, nodetype_test, lanes_df_train, lanes_df_test, meanspeed_train, meanspeed_test, z_score_train, z_score_test,congestion_train,congestion_test = train_test_split(n2v_start, n2v_end, lanes,oneway,roadtype ,maxspeed,nodetype,lanes_df,meanspeed,z_score,congestion)

del n2v_start
del n2v_end
del lanes
del oneway
del roadtype
del maxspeed
del nodetype
del lanes_df
del meanspeed
del z_score
del congestion

In [11]:
embinplist = list()
inplist = list()

n2v_start_inp = Input((128,))
n2v_start_reg = Dense(128,activation = "relu", use_bias = True)(n2v_start_inp)
inplist.append(n2v_start_inp)
embinplist.append(n2v_start_reg)

n2v_end_inp = Input((128,))
n2v_end_reg = Dense(128,activation = "relu", use_bias = True)(n2v_end_inp)
inplist.append(n2v_end_inp)
embinplist.append(n2v_end_reg)

lanes_inp = Input((1,))
lanes_reg= Dense(1,activation = "relu", use_bias = True)(lanes_inp)
inplist.append(lanes_inp)
embinplist.append(lanes_reg)

oneway_inp = Input((2,))
oneway_reg= Dense(2,activation = "relu", use_bias = True)(oneway_inp)
inplist.append(oneway_inp)
embinplist.append(oneway_reg)

roadtype_inp = Input((13,))
roadtype_reg= Dense(8,activation = "relu", use_bias = True)(roadtype_inp)
inplist.append(roadtype_inp)
embinplist.append(roadtype_reg)

maxspeed_inp = Input((1,))
maxspeed_reg= Dense(1,activation = "relu", use_bias = True)(maxspeed_inp)
inplist.append(maxspeed_inp)
embinplist.append(maxspeed_reg)

nodetype_inp = Input((11,))
nodetype_reg= Dense(8,activation = "relu", use_bias = True)(nodetype_inp)
inplist.append(nodetype_inp)
embinplist.append(nodetype_reg)

lanes_df_inp = Input((1,))
lanes_df_reg= Dense(1,activation = "relu", use_bias = True)(lanes_df_inp)
inplist.append(lanes_df_inp)
embinplist.append(lanes_df_reg)

combine = Concatenate()(embinplist)
proj = Dense(32,activation = "relu",use_bias = True)(combine)#proj = Dense(32,activation = "relu",use_bias = True)(combine)
proj = Dense(1,activation = "sigmoid")(proj)

emb = Model(inplist,proj)

In [4]:
emb.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 2)]          0                                            
______________________________________________________________________________________________

## probability

In [12]:
adam=Adam(lr=0.01,beta_1=0.9,beta_2=0.999,amsgrad=True)
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
emb.compile(optimizer=adam, loss='binary_crossentropy',metrics=['accuracy'])

In [13]:
history =emb.fit([n2v_start_train,n2v_end_train,lanes_train,oneway_train,roadtype_train,maxspeed_train,nodetype_train,lanes_df_train],
        congestion_train,
        epochs=100,batch_size=256,
        validation_data=([n2v_start_test,n2v_end_test,lanes_test,oneway_test,roadtype_test,maxspeed_test,nodetype_test,lanes_df_test],
        congestion_test)
       )

Train on 86992 samples, validate on 28998 samples
Epoch 1/100
86992/86992 [==============================] - 4s 47us/sample - loss: 0.3831 - acc: 0.8647 - val_loss: 0.3162 - val_acc: 0.8768
Epoch 2/100
86992/86992 [==============================] - 3s 40us/sample - loss: 0.2789 - acc: 0.8899 - val_loss: 0.2687 - val_acc: 0.8944
Epoch 3/100
86992/86992 [==============================] - 3s 39us/sample - loss: 0.2250 - acc: 0.9117 - val_loss: 0.2322 - val_acc: 0.9089
Epoch 4/100
86992/86992 [==============================] - 4s 40us/sample - loss: 0.1869 - acc: 0.9255 - val_loss: 0.2238 - val_acc: 0.9124
Epoch 5/100
86992/86992 [==============================] - 3s 39us/sample - loss: 0.1634 - acc: 0.9359 - val_loss: 0.2071 - val_acc: 0.9194
Epoch 6/100
86992/86992 [==============================] - 3s 40us/sample - loss: 0.1422 - acc: 0.9441 - val_loss: 0.2044 - val_acc: 0.9233
Epoch 7/100
86992/86992 [==============================] - 3s 39us/sample - loss: 0.1272 - acc: 0.9493 - val_l

Epoch 59/100
86992/86992 [==============================] - 3s 40us/sample - loss: 0.0144 - acc: 0.9955 - val_loss: 0.4436 - val_acc: 0.9347
Epoch 60/100
86992/86992 [==============================] - 4s 40us/sample - loss: 0.0125 - acc: 0.9960 - val_loss: 0.4310 - val_acc: 0.9324
Epoch 61/100
86992/86992 [==============================] - 3s 40us/sample - loss: 0.0097 - acc: 0.9970 - val_loss: 0.4577 - val_acc: 0.9355
Epoch 62/100
86992/86992 [==============================] - 4s 40us/sample - loss: 0.0082 - acc: 0.9976 - val_loss: 0.4417 - val_acc: 0.9348
Epoch 63/100
86992/86992 [==============================] - 3s 38us/sample - loss: 0.0079 - acc: 0.9977 - val_loss: 0.4649 - val_acc: 0.9359
Epoch 64/100
86992/86992 [==============================] - 3s 40us/sample - loss: 0.0075 - acc: 0.9978 - val_loss: 0.4685 - val_acc: 0.9351
Epoch 65/100
86992/86992 [==============================] - 3s 39us/sample - loss: 0.0066 - acc: 0.9980 - val_loss: 0.4760 - val_acc: 0.9361
Epoch 66/100


In [49]:
emb.save('/home/fyp2/Ideaer/Madrid/model/all')

In [21]:
loss_hist = history.history['acc']
mae_history = history.history["val_acc"]
print("max acc :"+str(np.max(loss_hist)))
print("max val_acc :"+str(np.max(mae_history)))

max acc :0.99968964
max val_acc :0.9380992


In [15]:
loss_hist = history.history['loss']
acc_history = history.history["val_acc"]
print("min loss :"+str(np.min(loss_hist)))
print("max validation accuarcy :"+str(np.max(acc_history)))

min loss :0.0010945477232417334
max validation accuarcy :0.9380992


## z-score

In [18]:
embinplist = list()
inplist = list()

n2v_start_inp = Input((128,))
n2v_start_reg = Dense(128,activation = "relu", use_bias = True)(n2v_start_inp)
inplist.append(n2v_start_inp)
embinplist.append(n2v_start_reg)

n2v_end_inp = Input((128,))
n2v_end_reg = Dense(128,activation = "relu", use_bias = True)(n2v_end_inp)
inplist.append(n2v_end_inp)
embinplist.append(n2v_end_reg)

lanes_inp = Input((1,))
lanes_reg= Dense(1,activation = "relu", use_bias = True)(lanes_inp)
inplist.append(lanes_inp)
embinplist.append(lanes_reg)

oneway_inp = Input((2,))
oneway_reg= Dense(2,activation = "relu", use_bias = True)(oneway_inp)
inplist.append(oneway_inp)
embinplist.append(oneway_reg)

roadtype_inp = Input((13,))
roadtype_reg= Dense(8,activation = "relu", use_bias = True)(roadtype_inp)
inplist.append(roadtype_inp)
embinplist.append(roadtype_reg)

maxspeed_inp = Input((1,))
maxspeed_reg= Dense(1,activation = "relu", use_bias = True)(maxspeed_inp)
inplist.append(maxspeed_inp)
embinplist.append(maxspeed_reg)

nodetype_inp = Input((11,))
nodetype_reg= Dense(8,activation = "relu", use_bias = True)(nodetype_inp)
inplist.append(nodetype_inp)
embinplist.append(nodetype_reg)

lanes_df_inp = Input((1,))
lanes_df_reg= Dense(1,activation = "relu", use_bias = True)(lanes_df_inp)
inplist.append(lanes_df_inp)
embinplist.append(lanes_df_reg)

combine = Concatenate()(embinplist)
proj = Dense(32,activation = "relu",use_bias = True)(combine)#proj = Dense(32,activation = "relu",use_bias = True)(combine)
proj = Dense(1,activation = "linear")(proj)

emb = Model(inplist,proj)

In [19]:
adam=Adam(lr=0.01,beta_1=0.9,beta_2=0.999,amsgrad=True)
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
emb.compile(optimizer=adam, loss='mae',metrics=['mae'])

In [20]:
history =emb.fit([n2v_start_train,n2v_end_train,lanes_train,oneway_train,roadtype_train,maxspeed_train,nodetype_train,lanes_df_train],
        z_score_train,
        epochs=100,batch_size=256,
        validation_data=([n2v_start_test,n2v_end_test,lanes_test,oneway_test,roadtype_test,maxspeed_test,nodetype_test,lanes_df_test],
        z_score_test)
       )

Train on 86992 samples, validate on 28998 samples
Epoch 1/100
86992/86992 [==============================] - 4s 47us/sample - loss: 0.7707 - mean_absolute_error: 0.7707 - val_loss: 0.7687 - val_mean_absolute_error: 0.7687
Epoch 2/100
86992/86992 [==============================] - 3s 39us/sample - loss: 0.7602 - mean_absolute_error: 0.7602 - val_loss: 0.7687 - val_mean_absolute_error: 0.7687
Epoch 3/100
86992/86992 [==============================] - 3s 39us/sample - loss: 0.7603 - mean_absolute_error: 0.7603 - val_loss: 0.7689 - val_mean_absolute_error: 0.7689
Epoch 4/100
86992/86992 [==============================] - 3s 38us/sample - loss: 0.7603 - mean_absolute_error: 0.7603 - val_loss: 0.7691 - val_mean_absolute_error: 0.7691
Epoch 5/100
86992/86992 [==============================] - 3s 38us/sample - loss: 0.7603 - mean_absolute_error: 0.7603 - val_loss: 0.7695 - val_mean_absolute_error: 0.7695
Epoch 6/100
86992/86992 [==============================] - 3s 38us/sample - loss: 0.7602 -

KeyboardInterrupt: 